In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: / 

In [ ]:
CLIENT_ID = 'KT4XBQ22N4LEKN42EZKB4RAIQC2REPNTQRDAE3KNYBBETLDG' # your Foursquare ID
CLIENT_SECRET = 'MMP2DX5WXCQZD1B2ZH4RJM5GEAMU3RRBTP0QPDYEMQZ04TRM' # your Foursquare Secret
ACCESS_TOKEN = 'VFS2C0IXHMQRDDWQFUTBNAOKNCZE0JARR1XMUQVN2MYR5TOU' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [ ]:
!pip install geocoder
import geocoder # import geocoder

In [ ]:
# Latitude & Longitude input
i=0
df ['Borough'] = ""
df ['Neighborhood'] = ""
geolocator = Nominatim(user_agent="foursquare_agent")

for i in range (df.shape[0]):
  Latitude = str(df.iloc[i,1])
  Longitude = str(df.iloc[i,2])
  print (Latitude, Longitude)
  lala = geolocator.reverse(Latitude+","+Longitude)
  #print (lala)
  location = str(lala)
  arrLoc = location.split(",")
  print (arrLoc[-6])
  print (arrLoc[1])
  df.loc [i, 'Borough'] = arrLoc[-6]
  df.loc [i, 'Neighborhood'] = arrLoc[1]

df = df[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df.head()

In [ ]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [ ]:
# type your answer here
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    '43.6532', 
    '-79.3832', 
    radius, 
    LIMIT)
url # display URL

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
Toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [ ]:
print(Toronto_venues.shape)
Toronto_venues.head()

In [ ]:
Toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

In [ ]:
Toronto_onehot.shape

In [ ]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

In [ ]:
i=0
contractor_map = folium.Map(location=[latitude, longitude], zoom_start=13) 
x = np.arange(manhattan_data.shape[0])
ys = [i + x + (i*x)**2 for i in range(manhattan_data.shape[0])]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for i in range (manhattan_data.shape[0]):
    radius = 500 # define radius

    # create URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, 
      CLIENT_SECRET,
      manhattan_data.loc[i, 'Latitude'],
      manhattan_data.loc[i, 'Longitude'],
      ACCESS_TOKEN,
      VERSION,
      search_query,
      radius,
      LIMIT)

    results = requests.get(url).json()['response']

    if len(results) > 0:
      venues = results['venues']

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    print ('The number of {} is {}'.format(manhattan_data.loc[i, 'Neighborhood'],dataframe.shape[0]))
    #dataframe.head()

    if dataframe.shape[0] > 0:
      filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
      df_Contractor = dataframe.loc[:, filtered_columns]

      # function that extracts the category of the venue
      def get_category_type(row):
          try:
              categories_list = row['categories']
          except:
              categories_list = row['venue.categories']
              
          if len(categories_list) == 0:
              return None
          else:
              return categories_list[0]['name']

      # filter the category for each row
      df_Contractor['categories'] = df_Contractor.apply(get_category_type, axis=1)

      # clean column names by keeping only last term
      df_Contractor.columns = [column.split('.')[-1] for column in df_Contractor.columns]

      # add a red circle marker to represent the Conrad Hotel
      folium.CircleMarker(
          [latitude, longitude],
          radius=10,
          color='red',
          popup='Conrad Hotel',
          fill = True,
          fill_color = 'red',
          fill_opacity = 0.6
      ).add_to(contractor_map)

      # add the Italian restaurants as blue circle markers
      for lat, lng, label in zip(df_Contractor.lat, df_Contractor.lng, df_Contractor.categories):
          folium.CircleMarker(
              [lat, lng],
              radius=5,
              color=rainbow[i],
              popup=label,
              fill = True,
              fill_color=rainbow[i],
              fill_opacity=0.6
          ).add_to(contractor_map)

contractor_map